# Machine Learning with Spark
## Linear Regression

In [1]:
import findspark
findspark.init('/home/fernando/spark-2.4.6-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('Linear_Regression/sample_linear_regression_data.txt')

In [5]:
training.count()

501

In [6]:
training.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                501|
|   mean|0.25688882219498976|
| stddev| 10.317884030544564|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [8]:
lr = LinearRegression(
    featuresCol='features',
    labelCol='label',
    predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

variance

In [13]:
training_summary.r2

0.027839179518600154

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

# Split the data, realistic situation

In [17]:
all_data = spark.read.format('libsvm').load('Linear_Regression/sample_linear_regression_data.txt')

In [18]:
train_data, test_data = all_data.randomSplit([0.7, 0.3]) # returns a list of 0.7 and 0.3 of data

In [19]:
train_data.count()

367

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                367|
|   mean| 0.5928663968449459|
| stddev| 10.351156581395912|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [21]:
test_data.count()

134

In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                134|
|   mean|-0.6632885650925822|
| stddev| 10.207942805253827|
|    min| -23.51088409032297|
|    max|  22.31738046492344|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

In [24]:
test_results = correct_model.evaluate(test_data)

In [26]:
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-22.591867613743013|
|-24.364629832384885|
|-20.552188161811515|
| -21.82942969089097|
|-20.937299028572593|
+-------------------+
only showing top 5 rows



In [27]:
test_results.rootMeanSquaredError

10.447802005816603

In [28]:
unlabeled = test_data.select('features')

In [29]:
unlabeled.show(3)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 3 rows



In [30]:
preds = correct_model.transform(unlabeled)

In [31]:
preds.show(10)

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.9190164765799576|
|(10,[0,1,2,3,4,5,...|  0.8771897114483747|
|(10,[0,1,2,3,4,5,...| 0.34011090285284185|
|(10,[0,1,2,3,4,5,...|   1.771947075101757|
|(10,[0,1,2,3,4,5,...|   1.052738254299167|
|(10,[0,1,2,3,4,5,...|   1.725690146256292|
|(10,[0,1,2,3,4,5,...|    1.60606070617935|
|(10,[0,1,2,3,4,5,...|   2.716317145566891|
|(10,[0,1,2,3,4,5,...|  4.7872795960817065|
|(10,[0,1,2,3,4,5,...|-0.38217637955948414|
+--------------------+--------------------+
only showing top 10 rows

